In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import time

In [2]:
browser = Browser('chrome',executable_path='chromedriver', headless=True)

# Visit the NASA Mars News Site

In [3]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

browser.is_element_present_by_css('ul.item_list li.slide', wait_time=1)

True

In [4]:
html = browser.html
news_soup = bs(html,'html.parser')
slide_elem = news_soup.select_one('ul.item_list li.slide')

In [5]:
news_title=slide_elem.find('div', class_='content_title').get_text()
news_title

"10.9 Million Names Now Aboard NASA's Perseverance Mars Rover"

In [6]:
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

"As part of NASA's 'Send Your Name to Mars' campaign, they've been stenciled onto three microchips along with essays from NASA's 'Name the Rover' contest. Next stop: Mars."

# JPL Space Images Featured Image

In [7]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [8]:
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

In [9]:
browser.is_element_not_present_by_text('more info', wait_time=1)
more_info_elem = browser.links.find_by_partial_text('more info')
more_info_elem.click()

In [10]:
html=browser.html
img_soup = bs(html,"html.parser")

In [11]:
img_url_rel = img_soup.select("figure.lede a img")[0].get('src')
img_url_rel

'/spaceimages/images/largesize/PIA09178_hires.jpg'

In [12]:
img_url = f"https://www.jpl.nasa.gov{img_url_rel}"
img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA09178_hires.jpg'

# Mars Weather

In [13]:
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)

time.sleep(5)

In [14]:
html = browser.html
weather_soup = bs(html,'html.parser')

In [15]:
mars_weather_tweet = weather_soup.find('div', attrs={'class': 'tweet', 'data-name':'Mars Weather'})

In [16]:
try:
    mars_weather = mars_weather_tweet.find('p','tweet-text').get_text()
    mars_weather
    
except AttributeError:
    pattern = re.compile(r'sol')
    mars_weather = weather_soup.find('span', text=pattern).text
    mars_weather
    
mars_weather

AttributeError: 'NoneType' object has no attribute 'text'

# Mars Hemispheres

In [17]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [18]:
hemisphere_image_urls = []

In [19]:
links = browser.find_by_css('a.product-item h3')
links

In [24]:
for i in range(len(links)):
    hemisphere = {}
    
    browser.find_by_css('a .product-item')[i].click()
    sample_elem = browser.links.find_by_text('Sample').first
    hemisphere['img_url'] = sample_elem['href']
    hemisphere['title'] = browser.find_by_css('h2.title').text
    hemispere_image_urls.append(hemisphere)
    browser.back()

ElementDoesNotExist: no elements could be found with css "a .product-item"

# Mars Facts

In [25]:
import lxml

In [30]:
df = pd.read_html('https://space-facts.com/mars/')[0]
df.columns=['description','value']
df.set_index('description')

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [32]:
df.to_html('Mars_Facts.html')